In [ ]:
# Variables
GITHUB_TOKEN = "INSERT_GITHUB_TOKEN_HERE"
API_BASE = "https://api.github.com"
HEADERS = {
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "X-GitHub-Api-Version": "2022-11-28",
}

owner = "mirmozavr"
repo = "GH-API-Analyst-Test"
user = "python"

In [ ]:
!pip install requests

import requests
import time
from typing import Optional

In [ ]:
# Helper: Handle Rate Limiting
def handle_rate_limit(response):
    if response.status_code == 403 and "X-RateLimit-Remaining" in response.headers:
        reset_time = int(response.headers.get("X-RateLimit-Reset", 0))
        wait_time = max(0, reset_time - int(time.time()))
        print(f"Rate limit reached. Waiting {wait_time} seconds.")
        time.sleep(wait_time + 1)
        return True
    return False

# Helper: Paginated Request
def paginated_request(url: str, params: Optional[dict] = None):
    data = []
    page = 1
    params = params or {}
    while True:
        params["page"] = page
        response = requests.get(url, headers=HEADERS, params=params)
        while handle_rate_limit(response):
            response = requests.get(url, headers=HEADERS, params=params)
        if response.status_code != 200:
            print(f"Error: {response.status_code} - {response.text}")
            break
        page_data = response.json()
        if not page_data:
            break
        data.extend(page_data)
        page += 1
    return data


Request 1: Get Repository Content

In [ ]:
# Get Repository Content
url = f"{API_BASE}/repos/{owner}/{repo}/contents"
params = {"ref": "master"}

response = requests.get(url, headers=HEADERS, params=params)
while handle_rate_limit(response):
    response = requests.get(url, headers=HEADERS, params=params)

if response.status_code == 200:
    print("Sample Response:")
    print(response.json())
else:
    print(f"Error: {response.status_code} - {response.text}")


Request 2: Download Repository Archive (ZIP)

In [ ]:
# Cell 3: Download Repository Archive (ZIP)
url = f"{API_BASE}/repos/{owner}/{repo}/zipball/master"

response = requests.get(url, headers=HEADERS, stream=True)
while handle_rate_limit(response):
    response = requests.get(url, headers=HEADERS, stream=True)

if response.status_code == 200:
    with open(f"{repo}.zip", "wb") as f:
        for chunk in response.iter_content(chunk_size=1024):
            f.write(chunk)
    print(f"Repository archive downloaded as {repo}.zip")
else:
    print(f"Error: {response.status_code} - {response.text}")


Request 3: List Repositories for Authenticated User

In [ ]:
url = f"{API_BASE}/user/repos"

repos = paginated_request(url)
print(f"Found {len(repos)} repositories:")
print(repos[:2])  # Print sample


Request 4: List Repositories for a User

In [ ]:
url = f"{API_BASE}/users/{user}/repos"

repos = paginated_request(url)
print(f"Found {len(repos)} repositories for user {user}:")
print(repos[:2])  # Print sample


Request 5: Search Repositories

In [ ]:
query = "pytorch+language:C"
params = {
    "q": query,
    "sort": "stars",
    "order": "desc",
    "per_page": 5,
    "page": 1,
}
url = f"{API_BASE}/search/repositories"

response = requests.get(url, headers=HEADERS, params=params)
while handle_rate_limit(response):
    response = requests.get(url, headers=HEADERS, params=params)

if response.status_code == 200:
    results = response.json()
    print("Sample Search Results:")
    print(results)
    # print(results["items"][:2])  # Print top 2 results
else:
    print(f"Error: {response.status_code} - {response.text}")


Request 6: List Commits

In [ ]:
url = f"{API_BASE}/repos/{owner}/{repo}/commits"

commits = paginated_request(url)
print(f"Found {len(commits)} commits:")
print(commits[:2])  # Print sample


Request 7: Get a Commit

In [ ]:
commit_sha = "master"  # Replace with a valid commit SHA
url = f"{API_BASE}/repos/{owner}/{repo}/commits/{commit_sha}"

response = requests.get(url, headers=HEADERS)
while handle_rate_limit(response):
    response = requests.get(url, headers=HEADERS)

if response.status_code == 200:
    commit = response.json()
    print("Sample Commit Details:")
    print(commit)
else:
    print(f"Error: {response.status_code} - {response.text}")
